## Where are you likely going to travel based on your recent history

## Import the necessary packages

In [1]:
import sys
sys.path.append('/home/xavier/DIR.insight2018/')
from secret import API_KEY, API_SECRET
import flickrapi
import time
import datetime
from calendar import monthrange
import pickle
import pandas as pd
import numpy as np
import xml.etree.ElementTree

flickr = flickrapi.FlickrAPI(API_KEY, API_SECRET, format='parsed-json')
#photos = flickr.walk(text='vacation',
#                     tag_mode='all',
#                     tags='cntower',
#                     extras='description,date_upload,date_taken,geo,tags,machine_tags,o_dims,views,url_s,url_m,url_l,url_o',
#                     media='photos',
#                     safe_search=1,
#                     content_type=1,
#                     per_page=50,
#                     has_geo=1,
#                     lat=43.6532,
#                     lon=-79.3832,
#                     radius=10)

#photos.__next__()
#x=photos.__next__()

In [30]:
#x.items()

In [2]:
#flickr.people.getInfo(user_id=flickr.people.findByUsername("APHendry1"))
x=flickr.people.findByUsername(username="APHendry1")

In [3]:
x['user']['nsid']

'83007471@N04'

In [33]:
#pge=1
#max_per_page_flickr = 500
#x=flickr.photos.search(tags="travel",per_page=max_per_page_flickr,page=pge)

In [34]:
#len(x['photos']['photo'])

## Deal with min and max date for the search

In [2]:
def unix_time_month_min(year,month):
    d = datetime.datetime(year,month,1,0,0,1)
    return int(time.mktime(d.timetuple()))
def unix_time_month_max(year,month):
    d = datetime.datetime(year,month,max(monthrange(year, month)),23,59,59)
    return int(time.mktime(d.timetuple()))
# [unix_time_month_min(2015,1),unix_time_month_max(2015,1)]

## Make a list of 20000 owner who use the tag 'travel'

In [43]:
## gave a 502 error
#list_user = {}
#max_per_page_flickr = 500
#for kw in ["travel", "vacation", "vacations", "holidays", "holiday", "travelling", "traveling","trip","sightseeing"]:
#    for y in range(2008,2018):
#        list_user["string{0}".format(y)]=[]
#        for m in range(1,13):
#            for pge in range(1,5):
#                x=flickr.photos.search(tags=kw,per_page=max_per_page_flickr,page=pge,
#                                       min_taken_date=unix_time_month_min(y,m),
#                                       max_taken_date=unix_time_month_max(y,m))
#                for i in range(len(x['photos']['photo'])):
#                    list_user["string{0}".format(y)].append(x['photos']['photo'][i]['owner'])

## Save the raw database to file

In [3]:
def clean_raw_db():
    raw_photos = pd.DataFrame(columns=['datetaken','description','latitude','longitude','owner','place_id','tags','title'])
    return raw_photos

def save_raw(n, raw_photos):
    file_n='raw_photos_'+str(n)+'.plk'
    with open(file_n, 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(raw_photos, f)
    return

In [ ]:
max_per_page_flickr = 500
raw_photos =  clean_raw_db()
total_comb = 9*10*12*4
raw_photos = clean_raw_db()
error_log_call = pd.DataFrame(columns=['keyword','year','month','page'])
a=0
startTime = datetime.datetime.now()
for y in range(2008,2019):
    for kw in ["travel", "vacation", "vacations", "holidays", "holiday", "travelling", "traveling","trip","sightseeing"]:
        for m in range(1,13):
            for pge in range(1,5):
                a+=1
                try:
                    x=flickr.photos.search(tags=kw,per_page=max_per_page_flickr,page=pge,
                                       min_taken_date=unix_time_month_min(y,m),
                                       max_taken_date=unix_time_month_max(y,m),
                                       extras='description,date_upload,date_taken,geo,tags')
                    if len(x['photos']['photo']) > 1:
                        raw_photos = raw_photos.append(pd.DataFrame(x['photos']['photo'])
                                   [['datetaken','description','latitude','longitude','owner',
                                     'place_id','tags','title']],ignore_index=True)
                    if a % 10 == 0 : 
                        print(a/total_comb, '  in ', datetime.datetime.now()-startTime)
                        print(a, " calls in ", datetime.datetime.now()-startTime)
                except:
                    print('Exception reached in photo', kw, y, m, pge)
                    error_log_call = error_log_call.append(pd.DataFrame([[kw,y,m,pge]],columns=['keyword','year','month','page']),ignore_index=True)
                    pass   # This just means continue
    save_raw(y, raw_photos) # save a different file for every year
    del raw_photos
    raw_photos = clean_raw_db()
#                x=flickr.walk(tags=kw,per_page=max_per_page_flickr,extras='description,date_upload,date_taken,geo') #,
#                                       min_taken_date=unix_time_month_min(y,m),
#                                       max_taken_date=unix_time_month_max(y,m)) 
#                for i in range(len(x['photos']['photo'])):
#                    list_user.append(x['photos']['photo'][i]['owner'])
#                    year.append(y)
#                    month.append(m)
#                    photo_id.append(x['photos']['photo'][i]['id'])
#                    tmp=pd.DataFrame({'user':pd.Series([x['photos']['photo'][i]['owner']], dtype='str'),
#                                      'year':pd.Series([y], dtype='int'),
#                                      'month':pd.Series([m], dtype='int')})
#                    list_user=list_user.append(tmp,ignore_index=True)
print(datetime.datetime.now() - startTime)
# Roughly 4:20 hours run time

0.0023148148148148147   in  0:00:19.010219
10  calls in  0:00:19.011578
0.004629629629629629   in  0:00:38.426500
20  calls in  0:00:38.426636
0.006944444444444444   in  0:00:58.077399
30  calls in  0:00:58.077537
0.009259259259259259   in  0:01:18.209622
40  calls in  0:01:18.209787
0.011574074074074073   in  0:01:37.434393
50  calls in  0:01:37.434535
0.013888888888888888   in  0:01:53.483694
60  calls in  0:01:53.483857
0.016203703703703703   in  0:02:10.389958
70  calls in  0:02:10.390111
0.018518518518518517   in  0:02:26.000759
80  calls in  0:02:26.001327
0.020833333333333332   in  0:02:43.042125
90  calls in  0:02:43.042270
0.023148148148148147   in  0:02:57.065570
100  calls in  0:02:57.065817
0.02546296296296296   in  0:03:06.560078
110  calls in  0:03:06.560220


In [15]:
kw

NameError: name 'kw' is not defined

In [18]:
error_log_call = pd.DataFrame(columns=['keyword','year','month','page'])
kw='a'
y=3
m=12
pge=3
tmp = pd.DataFrame([[kw,y,m,pge]],columns=['keyword','year','month','page'])
error_log_call
tmp

error_log_call = error_log_call.append(tmp,ignore_index=True)

m=6
tmp = pd.DataFrame([[kw,y,m,pge]],columns=['keyword','year','month','page'])
error_log_call = error_log_call.append(tmp,ignore_index=True)
error_log_call

,keyword,year,month,page
0,a,3,12,3
1,a,3,6,3


In [16]:
## sandbox to find geotag and location
#pd.DataFrame(x['photos']['photo'])[['datetaken','description','latitude','longitude','owner','place_id','tags','title']]
#pd.DataFrame(flickr.photos.getInfo(photo_id='43933280134')['photo']['tags']['tag'])['raw'].values.tolist()

In [6]:
with open('userList.plk', 'rb') as f:  # Python 3: open(..., 'rb')
    raw_photos =  pickle.load(f)

## Transform lists to a sorted panda data frame

In [17]:
tmp = {'User': list_user,
       'Month': month,
       'Year': year}
raw_user = pd.DataFrame(tmp)
raw_user.sort_values(by=['User'],inplace=True) # stort the list
raw_user.reset_index(drop=True,inplace=True)
#raw_user

,User,Month,Year
0,100001528@N06,7,2016
1,100001528@N06,7,2016
2,100001528@N06,7,2016
3,100001528@N06,7,2016
4,100001528@N06,7,2016
5,100001528@N06,7,2016
6,100001528@N06,7,2016
7,100001528@N06,7,2016
8,100001930@N07,6,2010
9,100002077@N08,4,2013


## Pick unique users who are present in more than one year


In [20]:
unique_user = {}
#a=0
#startTime = datetime.datetime.now()
# First entry
tmp_user=raw_user['User'][0]
tmp_list_year=[raw_user['Year'][0]]
n_row=len(raw_user['User'])
for i in range(1,n_row): # surf on the list grouped by user faster than doing search df['user'= user]
#    a+=1
    if tmp_user == raw_user['User'][i] :
        tmp_list_year.append(raw_user['Year'][i])
    else:
        set_tmp_list_year = set(tmp_list_year)
        if len(set_tmp_list_year) >1:
            unique_user[tmp_user] = set_tmp_list_year
        # Reset
        tmp_user=raw_user['User'][i]
        tmp_list_year=[raw_user['Year'][i]]
with open('unique_users.plk', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([unique_user], f)
#    if a % 1000 ==0 : print(a/n_row, 'in',  datetime.datetime.now()-startTime)
#set_tmp_list_year = set(tmp_list_year)
#if len(set_tmp_list_year) > 1:
#    unique_user[tmp_user] = set_tmp_list_year
## This is a 1min20 seconds process VS 4 hours for the procedure below

#unique_list = set(raw_user['User'])
#max_usr=len(unique_list)
#for user in unique_list:
#    x=set(raw_user[raw_user['User'] == user]['Year'])
#    length_x = len(x)
#    a +=1
#    if length_x > 1:
#        unique_user.update({'User': i,
#                            'n_years': length_x,
#                            'years': x})
#        
#    if a % 10 ==0 : print(a/max_usr, 'in',  datetime.datetime.now()-startTime)

In [23]:
with open('unique_user.plk', 'rb') as f:  # Python 3: open(..., 'rb')
    unique_user =  pickle.load(f)

In [22]:
raw_user.tail

<bound method NDFrame.tail of                   User  Month  Year
0        100001528@N06      7  2016
1        100001528@N06      7  2016
2        100001528@N06      7  2016
3        100001528@N06      7  2016
4        100001528@N06      7  2016
5        100001528@N06      7  2016
6        100001528@N06      7  2016
7        100001528@N06      7  2016
8        100001930@N07      6  2010
9        100002077@N08      4  2013
10       100002077@N08      4  2013
11       100002077@N08      4  2013
12       100002077@N08      4  2012
13       100002077@N08      4  2012
14       100002077@N08      4  2012
15       100002077@N08      4  2013
16       100002077@N08      4  2013
17       100002797@N05     10  2013
18       100002797@N05     10  2013
19       100005349@N03      5  2015
20       100005349@N03      8  2015
21       100005349@N03      8  2015
22       100005349@N03      8  2015
23       100005349@N03      8  2015
24       100005349@N03      8  2015
25       100005349@N03      8  201

In [16]:
len(list_user)

2000

In [57]:
[len(list_user) , len(set(list_user))]
uniqueUsers=set(list_user)
len(uniqueUsers)

88167

## Save the list of user who use the previous tag

In [116]:
with open('userList.plk', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([list_user,year,month], f)

## Getting back the objects:

In [10]:
with open('userList.plk', 'r') as f:  # Python 3: open(..., 'rb')
   tmp =  pickle.load(f)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [24]:
##sandbox
d={}
for x in range(1,10):
        d["string{0}".format(x)]=[]
        

In [25]:
d

{'string1': [],
 'string2': [],
 'string3': [],
 'string4': [],
 'string5': [],
 'string6': [],
 'string7': [],
 'string8': [],
 'string9': []}

In [26]:
d['string1'].append("bye")

In [76]:

list_user

,user,year,month
0,24391862@N06,2017,1
1,134867221@N07,2017,1
2,23985194@N06,2017,1
3,157852789@N03,2017,1
4,86855701@N08,2017,1
5,72616463@N00,2017,1
6,159251252@N07,2017,1
7,159251252@N07,2017,1
8,159251252@N07,2017,1
9,159251252@N07,2017,1


In [112]:
datetime.datetime.now()


datetime.datetime(2018, 9, 16, 22, 59, 37, 856464)

In [106]:
x.append('allo')

In [114]:
list_user

['86423215@N00',
 '141836629@N07',
 '41119009@N00',
 '34690494@N06',
 '159615760@N04',
 '159615760@N04',
 '159615760@N04',
 '159615760@N04',
 '159615760@N04',
 '159615760@N04',
 '159615760@N04',
 '141836629@N07',
 '142710016@N02',
 '142710016@N02',
 '142710016@N02',
 '142710016@N02',
 '141487081@N02',
 '141487081@N02',
 '61456446@N06',
 '61456446@N06',
 '61456446@N06',
 '61456446@N06',
 '61456446@N06',
 '61456446@N06',
 '61456446@N06',
 '61456446@N06',
 '27004296@N03',
 '27964309@N06',
 '159544363@N04',
 '141487081@N02',
 '133926410@N03',
 '7156765@N05',
 '8398907@N02',
 '41119009@N00',
 '41119009@N00',
 '41119009@N00',
 '37041050@N03',
 '48284565@N04',
 '48284565@N04',
 '48489192@N06',
 '48489192@N06',
 '41119009@N00',
 '41119009@N00',
 '41119009@N00',
 '41119009@N00',
 '7156765@N05',
 '21728312@N03',
 '21728312@N03',
 '32852605@N03',
 '41119009@N00',
 '17904502@N00',
 '7156765@N05',
 '145472109@N04',
 '145472109@N04',
 '145472109@N04',
 '144005928@N02',
 '35232195@N06',
 '62887986@N0